<a href="https://colab.research.google.com/github/nadavmoyal/Final_Project-FetalCns/blob/main/Fetal_Cns_Final_reg_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
# Part 1: Imports and Mounting Drive
from google.colab import drive
drive.mount('/content/drive')
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

# Part 2: Define CustomDataset Class and Read CSV File
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)

        label = self.data_frame.iloc[idx, 2]  # Assuming the third column is the label

        return image, label

# Read the list of images from the CSV file
train_csv = pd.read_csv("/content/drive/MyDrive/FinalProject/training_set_pixel_size_and_HC.csv")

# Part 3: Define Transforms and Create Dataset
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

train_dataset = CustomDataset(csv_file="/content/drive/MyDrive/FinalProject/training_set_pixel_size_and_HC.csv",
                              root_dir="/content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/",
                              transform=transform)

# Part 4: Create DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Part 5: Define Your Model
class YourModel(nn.Module):
    def __init__(self, input_channels=1, input_size=64):
        super(YourModel, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        conv_output_size = input_size // 2 // 2
        self.fc1 = nn.Linear(32 * conv_output_size * conv_output_size, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = YourModel(input_channels=1, input_size=64)

# Part 6: Define Training Parameters and Optimizer
learning_rate = 0.001
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Part 7: Training Loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.float()  # Convert inputs to Float
        labels = labels.float()  # Convert labels to Float
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.float(), labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i  % 32  == 0:
            print(f'Epoch {epoch + 1}, Mini-batch {i + 1}, Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

# Part 8: Evaluation (Not included as it's not necessary for training)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1, Mini-batch 1, Loss: 368.7505
Epoch 2, Mini-batch 1, Loss: 57.9532
Epoch 3, Mini-batch 1, Loss: 41.3163
Epoch 4, Mini-batch 1, Loss: 43.9428
Epoch 5, Mini-batch 1, Loss: 31.7458
Epoch 6, Mini-batch 1, Loss: 50.3168
Epoch 7, Mini-batch 1, Loss: 35.5899
Epoch 8, Mini-batch 1, Loss: 54.8524
Epoch 9, Mini-batch 1, Loss: 33.4360
Epoch 10, Mini-batch 1, Loss: 21.9853


In [90]:
# Part 8: Save the trained model
model_path = "/content/drive/MyDrive/FinalProject/trained_model.pth"
torch.save(model.state_dict(), model_path)
print("Model saved successfully!")

Model saved successfully!


In [98]:
# Load the trained model
model_path = "/content/drive/MyDrive/FinalProject/trained_model.pth"
model = YourModel(input_channels=1, input_size=64)
model.load_state_dict(torch.load(model_path))
model.eval()

# Function to predict head circumference from a new image
def predict_head_circumference(image_path, model):
    # Open and preprocess the new image
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor()
    ])
    image = Image.open(image_path)
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    # Make prediction
    with torch.no_grad():
        output = model(image_tensor.float())

    return output.item()  # Return the predicted head circumference as a scalar

# Path for the new image
new_image_path = "/content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/001_HC.png"
# Call the function to predict head circumference from the new image
predicted_circumference = predict_head_circumference(new_image_path, model)
print("Predicted Head Circumference:", predicted_circumference)


Predicted Head Circumference: 184.13450622558594
